In [2]:
!pip install simple_image_download

  Using cached simple_image_download-0.5-py3-none-any.whl.metadata (6.2 kB)
INFO: pip is looking at multiple versions of simple-image-download to determine which version is compatible with other requirements. This could take a while.
  Using cached simple_image_download-0.4-py3-none-any.whl.metadata (2.4 kB)
  Using cached simple_image_download-0.2-py3-none-any.whl.metadata (494 bytes)
Using cached simple_image_download-0.2-py3-none-any.whl (3.8 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from simple_image_download import simple_image_download as simp

response = simp.simple_image_download

search_term = input("Enter the search term: ")
limit = int(input("Enter the number of images to download: "))

response().download(search_term, limit=limit)

HTTPSConnectionPool(host='market.uad.ac.id', port=443): Max retries exceeded with url: /wp-content/uploads/2022/01/WhatsApp-Image-2022-01-07-at-10.38.32-1.jpeg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7615d2b9f8b0>: Failed to establish a new connection: [Errno 111] Connection refused'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
HTTPSConnectionPool(host='market.uad.ac.id', port=443): Max retries exceeded with url: /wp-content/uploads/2022/01/WhatsApp-Image-2022-01-07-at-10.38.32-1.jpeg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7615d2b557c0>: Failed to establish a new connection: [Errno 111] Connection refused'))
('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
HTTPSConnectionPool(host='market.uad.ac.id', port=443): Max retries exceeded with url: /wp-content/uploads/2022/01/WhatsApp-Image-2022-01-07-at-10.38.32-1.jpeg (Caused by NewCo

KeyboardInterrupt: 

In [2]:
import torch

cireng_path = "/home/irman/Documents/Vision-Transformer-Intelligence-System-Project/simple_images/cireng"

# find len
import os

len(os.listdir(cireng_path))

174

In [3]:
import os
from torchvision import transforms
from PIL import Image
import torch
from pathlib import Path

def process_images(input_path, output_path=None):
    """
    Process all images in the input directory to 224x224 squares.
    
    Args:
        input_path (str): Path to input directory containing images
        output_path (str, optional): Path to output directory. If None, will create 'processed' subdirectory
    """
    # Create transform pipeline
    transform = transforms.Compose([
        transforms.Resize(256),  # Resize the shorter side to 256
        transforms.CenterCrop(224),  # Crop the center to 224x224
        transforms.ToTensor(),  # Convert to tensor
        transforms.ToPILImage()  # Convert back to PIL for saving
    ])
    
    # Setup input/output paths
    input_path = Path(input_path)
    if output_path is None:
        output_path = input_path.parent / (input_path.name + '_processed')
    else:
        output_path = Path(output_path)
    
    # Create output directory if it doesn't exist
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Process each image
    valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
    processed_count = 0
    skipped_count = 0
    
    for file_path in input_path.iterdir():
        if file_path.suffix.lower() in valid_extensions:
            try:
                # Open and process image
                with Image.open(file_path) as img:
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                    
                    # Apply transformations
                    processed_img = transform(img)
                    
                    # Save processed image
                    output_file = output_path / file_path.name
                    processed_img.save(output_file, quality=95)
                    processed_count += 1
                    
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
                skipped_count += 1
        else:
            skipped_count += 1
    
    return {
        'processed': processed_count,
        'skipped': skipped_count,
        'output_path': str(output_path)
    }

# Example usage
if __name__ == "__main__":
    cireng_path = "/home/irman/Documents/Vision-Transformer-Intelligence-System-Project/simple_images/cireng"
    results = process_images(cireng_path)
    print(f"Processed {results['processed']} images")
    print(f"Skipped {results['skipped']} files")
    print(f"Output saved to: {results['output_path']}")

Processed 174 images
Skipped 0 files
Output saved to: /home/irman/Documents/Vision-Transformer-Intelligence-System-Project/simple_images/cireng_processed


In [4]:
cireng_processed_path = "/home/irman/Documents/Vision-Transformer-Intelligence-System-Project/simple_images/cireng_processed"

# find len
import os

len(os.listdir(cireng_processed_path))

165

In [5]:
import os
import random
import shutil
from pathlib import Path

def select_random_images(source_path, output_path, num_images=100, seed=42):
    """
    Randomly select and copy a specified number of images from source to output directory.
    
    Args:
        source_path (str): Path to source directory containing images
        output_path (str): Path to output directory for selected images
        num_images (int): Number of images to select
        seed (int): Random seed for reproducibility
    
    Returns:
        dict: Summary of the operation
    """
    # Set random seed for reproducibility
    random.seed(seed)
    
    # Convert paths to Path objects
    source_path = Path(source_path)
    output_path = Path(output_path)
    
    # Get list of all image files
    valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
    image_files = [f for f in source_path.iterdir() 
                  if f.suffix.lower() in valid_extensions]
    
    # Verify we have enough images
    if len(image_files) < num_images:
        raise ValueError(f"Source directory only contains {len(image_files)} images, "
                        f"cannot select {num_images} images")
    
    # Randomly select images
    selected_images = random.sample(image_files, num_images)
    
    # Create output directory if it doesn't exist
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Copy selected images
    copied_count = 0
    for img_path in selected_images:
        try:
            shutil.copy2(img_path, output_path / img_path.name)
            copied_count += 1
        except Exception as e:
            print(f"Error copying {img_path}: {e}")
    
    return {
        'total_source_images': len(image_files),
        'images_selected': num_images,
        'images_copied': copied_count,
        'output_path': str(output_path)
    }

# Example usage
if __name__ == "__main__":
    cireng_processed_path = "/home/irman/Documents/Vision-Transformer-Intelligence-System-Project/simple_images/cireng_processed"
    cireng_final_path = "/home/irman/Documents/Vision-Transformer-Intelligence-System-Project/simple_images/cireng_final"
    
    # Select 100 random images
    results = select_random_images(
        source_path=cireng_processed_path,
        output_path=cireng_final_path,
        num_images=100
    )
    
    print(f"Source directory contains: {results['total_source_images']} images")
    print(f"Selected and copied: {results['images_copied']} images")
    print(f"Output saved to: {results['output_path']}")
    
    # Verify the number of images in the final directory
    final_count = len(os.listdir(cireng_final_path))
    print(f"Final directory contains: {final_count} images")

Source directory contains: 165 images
Selected and copied: 100 images
Output saved to: /home/irman/Documents/Vision-Transformer-Intelligence-System-Project/simple_images/cireng_final
Final directory contains: 100 images
